# PyBADS Example 4: Noisy objective with user-provided noise estimates

In this example, we will show how to run PyBADS on a noisy target for which we can estimate the noise at each evaluation.

This notebook is Part 4 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [1]:
import numpy as np
from pybads.bads.bads import BADS

## 1. Problem setup

We assume you are already familiar with optimization of noisy targets with PyBADS, as described in the [previous notebook](./pybads_example_3_noisy_objective.ipynb).

Sometimes, you may be able to estimate the noise associated with *each* function evaluation, for example via bootstrap or more sophisticated estimation methods. If you can do that, it is highly recommended you do so and tell PyBADS by activating the `specify_target_noise` option.

In the user-specified noise case, the target function `fun` is expected to return *two* outputs: 
- the noisy estimate of the function at `x` (as usual); 
- the estimate of the *standard deviation* of the noisy evaluation at `x`.

In this toy example below, we know the standard deviation `sigma` by construction. Note that the function is *heteroskedastic*, that is, the noise depends on the input location `x`.

In [2]:
def noisy_sphere_estimated_noise(x,scale=1.0):
    """Quadratic function with heteroskedastic noise; also return noise estimate."""
    x_2d = np.atleast_2d(x)
    f = np.sum(x_2d**2, axis=1)
    sigma = scale*(1.0 + np.sqrt(f))
    y = f + sigma*np.random.normal(size=x_2d.shape[0])
    return y, sigma

x0 = np.array([[-3, -3]]);      # Starting point
lb = np.array([[-5, -5]])       # Lower bounds
ub = np.array([[5, 5]])         # Upper bounds
plb = np.array([[-2, -2]])      # Plausible lower bounds
pub = np.array([[2, 2]])        # Plausible upper bounds

options = {
    "uncertainty_handling": True,
    "specify_target_noise": True,
    "noise_final_samples": 100
}

## 2. Run the optimization

As usual, we run `bads` with the user-defined `options`.

In [3]:
bads = BADS(noisy_sphere_estimated_noise, x0, lb, ub, plb, pub, options=options)
optimize_result = bads.optimize()

Beginning optimization of a STOCHASTIC objective function (specified noise)

 Iteration f-count     E[f(x)]     SD[f(x)]     MeshScale     Method     Actions
     0         1      17.621154      5.242641      1.000000            
     0        33      -2.525896      5.242641      1.000000     Initial mesh       Initial points
     0        37      -2.525896      2.909325      0.500000     Refine grid       Train


c:\users\luigi\documents\github\gpyreg\gpyreg\covariance_functions.py:376: RuntimeWarning: invalid value encountered in add
  plausible_lower_bounds[i_nan] + plausible_upper_bounds[i_nan]


     1        45      -2.525896      2.909325      0.250000     Refine grid       Train
     2        46       1.028197      0.610765      0.250000     Successful search (('ES-wcm', 1))       
     2        47      -0.314407      0.824240      0.250000     Successful search (('ES-wcm', 1))       
     2        49      -0.674746      0.650137      0.250000     Successful search (('ES-ell', 1))       
     2        57      -0.674746      0.650137      0.125000     Refine grid       Train
     3        58      -0.226220      0.368633      0.125000     Successful search (('ES-ell', 1))       
     3        69      -0.226220      0.368633      0.062500     Refine grid       Train
     4        70       0.087989      0.315454      0.062500     Successful search (('ES-wcm', 1))       
     4        71       0.071722      0.391966      0.062500     Successful search (('ES-wcm', 1))       
     4        76      -0.006757      0.291402      0.062500     Successful search (('ES-ell', 1))       
 

## 3. Results and conclusions

As per noisy target optimization, `optimize_result['fval']` is the *estimated* function value at `optimize_result['x']`, here obtained by taking the *weighted* mean of the final sampled evaluations (each evaluation is weighted by its precision, or inverse variance).

The final samples can be found in `optimize_result['yval_vec']`, and their estimated standard deviation in `optimize_result['ysd_vec']`.

In [4]:
x_min = optimize_result['x']
fval = optimize_result['fval']
fsd = optimize_result['fsd']

print(f"BADS minimum at: x_min = {x_min.flatten()}, fval (estimated) = {fval:.4g} +/- {fsd:.2g}")
print(f"total f-count: {optimize_result['func_count']}, time: {round(optimize_result['total_time'], 2)} s")
print(f"final evaluations (shape): {optimize_result['yval_vec'].shape}")

BADS minimum at: x_min = [-0.09112008 -0.63206721], fval (estimated) = 0.3951 +/- 0.17
total f-count: 376, time: 10.12 s
final evaluations (shape): (100,)


We can also check the ground-truth value of the target function at the returned point once we remove the noise:

In [5]:
print(f"The true, noiseless value of f(x_min) is {noisy_sphere_estimated_noise(x_min,scale=0)[0][0]:.3g}.")

The true, noiseless value of f(x_min) is 0.408.


Compare this to the true global minimum of the sphere function at $\textbf{x}^\star = [0,0]$, where $f^\star = 0$.

Due to the elevated level of noise, we do not necessarily expect high precision in the solution

## Example 4: Full code

See [here](./src/pybads_example_4_user_provided_noise.py) for a Python file with the code used in this example, with no extra fluff.